Script to get the unicode and transliterations for Sumerian glyphs from Wikipedia

In [1]:
import json
import re

from   bs4 import BeautifulSoup
import pandas as pd
from   requests_html import HTMLSession

In [2]:
url = 'https://en.wikipedia.org/wiki/Cuneiform_(Unicode_block)'
session = HTMLSession()

In [3]:
def get_soup(url):
    session = HTMLSession()
    res = session.get(url)
    html = res.html.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
def extract_data(soup):
    table = soup.find('table', class_='wikitable')
    df = pd.read_html(str(table))[0]
    df.rename(columns={'Sign': 'Sign', 
                       'Code point': 'Unicode', 
                       '"Primary" transliteration': 'Transliteration',
                       'Borger(2003)': 'Borger_2003',
                       'Borget(1981)': 'Borger_1981', 
                       'Comments': 'Comments'},
              inplace=True)
    return df

In [5]:
def add_keystrokes(text):
    try:
        # tenû, gunû, šešig
        text = text.replace('ten\u00fb', '')\
            .replace('gun\u00fb', '')\
            .replace('\u0161e\u0161ig', '')\
            .replace('\u00c1', 'A')\
            .replace('\u1e2a', 'H')\
            .replace('\u0160', 'SH')
        text = re.sub(' [x/+] ', '.', text)
        pattern = ('(crossing|inverted|opposing|reversed|rotated|signs|'
                   'squared|three times|variant form|90°|\*)')
        text = re.sub(pattern, '', text)
        text = text.strip()
        text = re.sub(' ?\. ?', '.', text)
        text = text.replace(' ', '.')
        text = text.replace('..', '.')
        return text.lower()
    except AttributeError:
        print(text)
        return 'na'

In [6]:
soup = get_soup(url)
df = extract_data(soup)
df.loc[:, 'Keystrokes'] = df.Transliteration.apply(add_keystrokes)
df.sort_values('Keystrokes', inplace=True)

nan


In [7]:
df.head(30)

,Sign,Unicode,Transliteration,Borger_2003,Borger(1981),Comments,Keystrokes
0,𒀀,U+12000,A,839,579,NaN,a
1,𒀁,U+12001,A x A,845,583,EDURU,a.a
2,𒀂,U+12002,A x BAD,840,580,AGAM,a.bad
3,𒀃,U+12003,A x GAN2 tenû,NaN,NaN,NaN,a.gan2
4,𒀄,U+12004,A x ḪA,846,584; 587,ZAḪ3,a.ha
5,𒀅,U+12005,A x IGI,844,581,ÍR,a.igi
6,𒀆,U+12006,A x LAGAR gunû,843,582,NaN,a.lagar
7,𒀇,U+12007,A x MUŠ,842,NaN,NaN,a.mush
8,𒀈,U+12008,A x SAG,841,585,NaN,a.sag
9,𒀉,U+12009,A2,560,334,ID,a2


In [112]:
df[['Sign', 'Unicode', 'Transliteration', 'Keystrokes']].to_csv(
    '../data/sumerian_transliteration_unicode.csv', index=False)

In [113]:
with open('../data/sumerianTransliterationUnicode.json', 'w') as f:
    df[
        ['Sign', 'Unicode', 'Transliteration', 'Keystrokes']
    ].to_json(f, orient='records')

In [8]:
def row_to_json(row):
    out = {row.Keystrokes: {'unicode': row.Unicode, 'glyph': row.Sign}}
    return out

In [9]:
row_to_json(df.loc[0, :])

{'a': {'unicode': 'U+12000', 'glyph': '𒀀'}}

In [12]:
def make_transliterator(df):
    out = {}
    for _, row in df.iterrows():
        entry = row_to_json(row)
        out.update(entry)
    return out

In [14]:
transliterator = make_transliterator(df)
with open('../data/transliterator.json', 'w') as f:
    json.dump(transliterator, f)